In [47]:
import pandas as pd
import sys
import Levenshtein
from  textprepare import TextColumnPrepare
from Levenshtein import *

In [48]:
def readedits(fileerror, filecorrect, id_from, id_to, col_prep=TextColumnPrepare()):
    data = []
    with open(fileerror, 'r') as erf:
        with open(filecorrect, 'r') as cor:
            i = 0
            while i < id_to:
                if i >= id_from:
                    data.append([col_prep.prepare_string(erf.readline()), 
                                 col_prep.prepare_string(cor.readline())])
                else:
                    erf.readline()
                    cof.readline()
                i +=1
    return data

In [49]:
data = readedits('wiked.tok.err', 'wiked.tok.cor', 0, 1000)

In [50]:
data[150]

['radical reactions dont deal charged particles radicals',
 'radical reactions apos deal charged particles radicals']

In [60]:
def get_statistis_words(data, max_distance=3):
    res_words = []
    for i, line in enumerate(data):
        errsent = line[0].split()
        corsent = line[1].split()
        for errw in errsent:
            for corw in corsent:
                # Remove small words
                if (len(errw) < 2) or (len(corw) < 2):
                    continue
                # Remove digits
                if errw.isdigit() or corw.isdigit():
                    continue
                d = distance(errw, corw) 
                if (d > 0) and d < max_distance:
                    res_words.append([errw, corw])
    return res_words

In [61]:
pairs = get_statistis_words(data)

In [62]:
pairs

[['wikimedia', 'wikipedia'],
 ['textbook', 'textbooks'],
 ['wikipedia', 'wikimedia'],
 ['textbooks', 'textbook'],
 ['sciences', 'science'],
 ['sciences', 'science'],
 ['sciences', 'science'],
 ['science', 'sciences'],
 ['book', 'books'],
 ['books', 'book'],
 ['book', 'books'],
 ['books', 'book'],
 ['book', 'books'],
 ['books', 'book'],
 ['shirt', 'hirt'],
 ['shirt', 'hirt'],
 ['site', 'see'],
 ['see', 'site'],
 ['wikibooks', 'wikibook'],
 ['wikibook', 'wikibooks'],
 ['wikibook', 'wikibooks'],
 ['wikibooks', 'wikibook'],
 ['eperiences', 'experiences'],
 ['reacions', 'reactions'],
 ['reactions', 'reacions'],
 ['lot', 'top'],
 ['top', 'lot'],
 ['e1', 'sn1'],
 ['e1', 'e2'],
 ['sn1', 'e1'],
 ['e2', 'e1'],
 ['cyclohexane', 'cyclohexene'],
 ['cyclohexene', 'cyclohexane'],
 ['derivates', 'derivatives'],
 ['supstituted', 'substituted'],
 ['produces', 'product'],
 ['produces', 'product'],
 ['product', 'produces'],
 ['product', 'produces'],
 ['produces', 'product'],
 ['produces', 'product'],
 ['p